In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install flask-ngrok
!pip install git+https://github.com/facebookresearch/fastText.git

  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-0_d0mf57
  Running command git clone -q https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-0_d0mf57
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3086504 sha256=454b972492df7f644d7d9263c1991e91d309e2a9d7e8fd468da6659e116bff4b
  Stored in directory: /tmp/pip-ephem-wheel-cache-h1qkv2yh/wheels/69/f8/19/7f0ab407c078795bc9f86e1f6381349254f86fd7d229902355
Successfully built fasttext


In [3]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 412.3MB 31kB/s 
     |████████████████████████████████| 3.8MB 30.7MB/s 
     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 512kB 43.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=02a05cf3eb0f2728ae102d3fe0a48cbc8375f6e74ae1ba7db4ff22f501c902b4
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

In [4]:
import numpy as np
from numpy import argmax

In [5]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D, Lambda, Softmax
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [6]:
from keras.models import load_model
m = load_model("/content/drive/MyDrive/model/model-1.h5")






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [7]:
#get word embedding libary
import fasttext.util
print('Creating fastText...')
word2vec = fasttext.load_model('/content/drive/MyDrive/fast_text/cc.vi.300.bin')
print('fastText created!')

Creating fastText...
fastText created!


In [8]:
!pip install vncorenlp

     |████████████████████████████████| 2.7MB 2.5MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp37-none-any.whl size=2645936 sha256=6ccde0a69779e276c9b89937491e2b4c819c604b8dd763aaf8b39374924eeb18
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp


In [9]:
# get word embedding
from vncorenlp import VnCoreNLP
print('Creating token word...')
annotator = VnCoreNLP("/content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
print('token word created')

Creating token word...
token word created


In [10]:
# aphabet tag library
tags = {'pad': 0, 'LOCATION_HOMENUMBER': 1, 'LOCATION_STREET': 2, 'LOCATION_WARD': 3, 'LOCATION_DISTRICT': 4, 'LOCATION_PROVINCE': 5, 'LOCATION_COUNTRY': 6, 'LOCATION_POSTCODE': 7, 'LOCATION_SPECIAL': 8, 'LOCATION_NER': 9, 'OBJ': 10, 'OBJ_FEATURE': 11, 'PRE': 12, 'UNKNOW': 13 }
aphabet_tag = []
for key in tags:
  aphabet_tag.append(key)
# aphabet_tag = ['pad', 'LOCATION_HOMENUMBER', 'LOCATION_STREET', 'LOCATION_WARD', 'LOCATION_DISTRICT', 'LOCATION_PROVINCE', 'LOCATION_COUNTRY', 'LOCATION_POSTCODE', 'OBJ', 'OBJ_FEATURE', 'PRE', 'UNKNOW', 'LOCATION_SPECIAL']
print(aphabet_tag)

['pad', 'LOCATION_HOMENUMBER', 'LOCATION_STREET', 'LOCATION_WARD', 'LOCATION_DISTRICT', 'LOCATION_PROVINCE', 'LOCATION_COUNTRY', 'LOCATION_POSTCODE', 'LOCATION_SPECIAL', 'LOCATION_NER', 'OBJ', 'OBJ_FEATURE', 'PRE', 'UNKNOW']


In [11]:
def map_number_and_punct(word):
    # check hem va ngach
    num_of_seperate=0
    dem=0
    for char in word:
      if not char.isnumeric() and char!="/":
        dem+=1        
      if char=="/":
        num_of_seperate+=1
    if dem==0:
      if len(word)>1 and num_of_seperate==1:
        return u'<ngach>'
      if len(word)>1 and num_of_seperate>2:
        return u'<hem>'

    if re.match(r"^[0-9]{2}0{3}$", word):
        return u'<postcode>'

    if word.isnumeric():
        word = u'<number>'
    elif word in [u',', u'<', u'.', u'>', u'/', u'?', u'..', u'...', u'....', u':', u';', u'"', u"'", u'[', u'{', u']',
                  u'}', u'|', u'\\', u'`', u'~', u'!', u'@', u'#', u'$', u'%', u'^', u'&', u'*', u'(', u')', u'-', u'+',
                  u'=']:
        word = u'<punct>'

    return word

def get_word_embbeding(sen_words,max_length=32,embedd_dim = 300):
  X = np.empty([1, max_length, embedd_dim])
  unknown_embedd = np.random.uniform(-0.01, 0.01, [1, 300])
  length = len(sen_words)
  for i in range(length):
    try:
      vec = word2vec.get_word_vector(map_number_and_punct(sen_words[i])).tolist()
    except:
      vec = unknown_embedd
    X[0, i, :] = vec
  X[0,length:] = np.zeros([1, embedd_dim])
  return np.array(X)
# sen_words_embedd = get_word_embbeding(sen_words[0],max_length=25,embedd_dim = 300)
# print(sen_words_embedd.shape)

In [12]:
#get char embedding
def read_char_vocab(file_path):
  char_vocab = []
  for line in open(file_path, encoding='utf-8'):
    char_vocab.append(line.splitlines()[0])      
  return char_vocab

def char_to_int(char_vocab_data):
  dic = {}
  index = -1
  for char in char_vocab_data:
    try:
      dic[char]
    except:
      index = index + 1
      dic[char]=index
  return dic
 
def get_char_encode(sentence,max_length_of_a_sentence,max_length_of_a_word):
  char_vocab_data = read_char_vocab("/content/drive/MyDrive/fast_text/VISCII_short.txt")
  dic = char_to_int(char_vocab_data)
  # sentence  words is a sentence | ['i','am','an']
  sentence_encoded = np.zeros([max_length_of_a_sentence,max_length_of_a_word]) # 25*25
  for j in range(len(sentence)):  
    word = sentence[j].lower()
    # integer_encoded = [char_to_int[char] for char in word]
    word_encoded = np.zeros(max_length_of_a_word)
    for k in range(len(word)):
      char = word[k]
      try:
        word_encoded[k]= dic[char]
      except:
        print("error : " + str(char)+" "+str(word)+" "+str(len(word)))
        word_encoded[k]= dic['[unk]']
    # sentence encoded
    sentence_encoded[j] = word_encoded
  return sentence_encoded

def get_charEmbedd_form_encode(charEnocde):
  LEN_OF_VOCAB = 137
  shape = charEnocde.shape
  char_embedd = np.zeros([shape[0],shape[1],LEN_OF_VOCAB])
  for i in range(shape[0]):
    for j in range(shape[1]):
      char_int = charEnocde[i,j]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      char_embedd[i,j,:] = onehot
  return char_embedd

# return onehot_encoded_of_a_sentence

In [13]:
!unzip /content/drive/MyDrive/ngrok/ngrok-stable-linux-amd64.zip

Archive:  /content/drive/MyDrive/ngrok/ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [14]:
!./ngrok authtoken 1ow8g2aFZ4wILzg2dqHin5bUy5t_34f12x5zn89tGiLLCi4Wy

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [15]:
# query = "số nhà 103 - A12"
# # query = "tìm các nơi tổ chức tang lễ đang mở cửa xung quanh khu này"
# try:
#   sen_words = annotator.tokenize(query)[0]
#   num_of_word = len(sen_words)
# except:
#   print('Creating token word...')
#   annotator = VnCoreNLP("/content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
#   print('token word created')
#   sen_words = annotator.tokenize(query)[0]
#   num_of_word = len(sen_words)
# print(sen_words)

['số', 'nhà', '103', '-', 'A12']


In [19]:
# # query = "tìm kiếm khách sạn 5 sao xung quanh chỗ này"
# # query = "sân bay đang mở cửa xung quanh 144 lô 14a trung hòa cầu giấy hà nội việt nam"
# # query = "tìm các nơi tổ chức tang lễ đang mở cửa xung quanh khu này"
# query = "14a ngõ 2 đức diễn kiều mai bắc từ liêm hà nội"
# try:
#   sen_words = annotator.tokenize(query)[0]
#   num_of_word = len(sen_words)
# except:
#   print('Creating token word...')
#   annotator = VnCoreNLP("/content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
#   print('token word created')
#   sen_words = annotator.tokenize(query)[0]
#   num_of_word = len(sen_words)
# print(sen_words)
    
# # get word embedding
# sen_words_embedd = get_word_embbeding(sen_words,max_length=42,embedd_dim = 300)
    
# # get char embedding
# char_encode = get_char_encode(sen_words,42,32)

# char_embedd = get_charEmbedd_form_encode(char_encode)
# char_embedd = char_embedd.reshape(1,char_embedd.shape[0], char_embedd.shape[1],char_embedd.shape[2])

# # predict
# output = m.predict([char_embedd,sen_words_embedd])

# # return [1,2,3,4,5,5,5,5,5,5,5,5,5,5,5] or convert to encode tag
# tag_encode = []
# for i in range(42): 
#   tag_encode.append(argmax(output[0,i,:]))

# # return [street, homenumber,.....] or only get [num_of_word] first element=> convert to tag
# tag_result=[]
# for i in range(num_of_word):
#   tag_result.append(aphabet_tag[tag_encode[i]])
#   arr_tag_result=np.array(tag_result)

# # return {'nhaf_hàng': OBJ,....} 
# result={}
# for i in range(num_of_word):
#   result[sen_words[i]] = str(arr_tag_result[i])

# print(result)
# #==========================
# dic_result = {}
# for word in result:
#   tag = result[word]
#   try:
#     dic_result[tag] += " " + word
#   except:
#     dic_result[tag] = word
# print(dic_result)

['14a', 'ngõ', '2', 'đức', 'diễn_kiều', 'mai', 'bắc', 'từ_liêm', 'hà_nội']
{'14a': 'LOCATION_HOMENUMBER', 'ngõ': 'LOCATION_STREET', '2': 'LOCATION_STREET', 'đức': 'LOCATION_STREET', 'diễn_kiều': 'LOCATION_STREET', 'mai': 'LOCATION_WARD', 'bắc': 'LOCATION_WARD', 'từ_liêm': 'LOCATION_PROVINCE', 'hà_nội': 'LOCATION_PROVINCE'}
{'LOCATION_HOMENUMBER': '14a', 'LOCATION_STREET': 'ngõ 2 đức diễn_kiều', 'LOCATION_WARD': 'mai bắc', 'LOCATION_PROVINCE': 'từ_liêm hà_nội'}


In [ ]:
import requests
#from flask import Flask, render_template, jsonify
from flask import Flask, render_template, abort, request, jsonify, json,Response
from flask import request, redirect, url_for
import codecs
import gensim
from distutils.version import LooseVersion, StrictVersion
import json
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)
app.config['JSON_AS_ASCII'] = False

global word2vec_model

@app.route('/')
def summary():
  if request.method == "GET":
    query = request.values['query'] or ''
    query = str(query)
    print( 'query = ' + query )
    
    # token word
    try:
      sen_words = annotator.tokenize(query)[0]
      num_of_word = len(sen_words)
    except:
      print('Creating token word...')
      annotator = VnCoreNLP("/content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
      print('token word created')
      sen_words = annotator.tokenize(query)[0]
      num_of_word = len(sen_words)
    print(sen_words)
        
    # get word embedding
    sen_words_embedd = get_word_embbeding(sen_words,max_length=42,embedd_dim = 300)
    print("sen_words_embedd done!")
    # get char embedding
    char_encode = get_char_encode(sen_words,42,32)

    char_embedd = get_charEmbedd_form_encode(char_encode)
    char_embedd = char_embedd.reshape(1,char_embedd.shape[0], char_embedd.shape[1],char_embedd.shape[2])
    print("char_embedd done!")
    # predict
    output = m.predict([char_embedd,sen_words_embedd])
    print("output done!")
    # return [1,2,3,4,5,5,5,5,5,5,5,5,5,5,5] or convert to encode tag
    tag_encode = []
    for i in range(42): 
      tag_encode.append(argmax(output[0,i,:]))

    # return [street, homenumber,.....] or only get [num_of_word] first element=> convert to tag
    tag_result=[]
    for i in range(num_of_word):
      tag_result.append(aphabet_tag[tag_encode[i]])
      arr_tag_result=np.array(tag_result)

    # return {'nhaf_hàng': OBJ,....} 
    result={}
    for i in range(num_of_word):
      result[sen_words[i]] = str(arr_tag_result[i])

    print(result)
    #==========================
    dic_result = {}
    for word in result:
      tag = result[word]
      try:
        dic_result[tag] += ", " + word
      except:
        dic_result[tag] = word
    print(dic_result)

    return jsonify(sen_words=sen_words, tags=tag_result, output=dic_result)
if __name__ == "__main__":
  import os
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f7362b018b58.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
query = "144 xuân thủy cầu giấy hà nội"
Creating token word...


127.0.0.1 - - [22/Apr/2021 04:16:35] "GET /?query=%22144%20xuân%20thủy%20cầu%20giấy%20hà%20nội%22 HTTP/1.1" 200 -


token word created
['"', '144', 'xuân', 'thuỷ', 'cầu_giấy', 'hà_nội', '"']
sen_words_embedd done!
char_embedd done!
output done!
{'"': 'pad', '144': 'LOCATION_STREET', 'xuân': 'LOCATION_WARD', 'thuỷ': 'LOCATION_WARD', 'cầu_giấy': 'LOCATION_DISTRICT', 'hà_nội': 'LOCATION_PROVINCE'}
{'pad': '"', 'LOCATION_STREET': '144', 'LOCATION_WARD': 'xuân, thuỷ', 'LOCATION_DISTRICT': 'cầu_giấy', 'LOCATION_PROVINCE': 'hà_nội'}


127.0.0.1 - - [22/Apr/2021 04:16:36] "GET /favicon.ico HTTP/1.1" 404 -


query = 144 xuân thủy cầu giấy hà nội
Creating token word...


127.0.0.1 - - [22/Apr/2021 04:21:30] "GET /?query=144%20xuân%20thủy%20cầu%20giấy%20hà%20nội HTTP/1.1" 200 -


token word created
['144', 'xuân', 'thuỷ', 'cầu_giấy', 'hà_nội']
sen_words_embedd done!
char_embedd done!
output done!
{'144': 'LOCATION_HOMENUMBER', 'xuân': 'LOCATION_WARD', 'thuỷ': 'LOCATION_WARD', 'cầu_giấy': 'LOCATION_DISTRICT', 'hà_nội': 'LOCATION_PROVINCE'}
{'LOCATION_HOMENUMBER': '144', 'LOCATION_WARD': 'xuân, thuỷ', 'LOCATION_DISTRICT': 'cầu_giấy', 'LOCATION_PROVINCE': 'hà_nội'}
